# ADK with MCP using BareHand: JSON-RPC

Caution: This may not be the right note book if you are looking for a quick start of MCP.

This note book is mainly for showing the internal works of MPC server at JSON-RPC layer.

Last Update:

- July 1, 2025
- ADK == 1.5.0
- Primary model: gemini-2.5-flash

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.sandbox.google.com/github/hupili/google-adk-101/blob/main/MCP_Barehand_JSON_RPC.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fhupili%2Fgoogle-adk-101%2Fmain%2FMCP_Barehand_JSON_RPC.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-enterprise-logo.png" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/hupili/google-adk-101/blob/main/MCP_Barehand_JSON_RPC.ipynb">
      <img width="32px" src="https://www.gstatic.com/monospace/240802/git_host_github_mask.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/hupili/google-adk-101/main/MCP_Barehand_JSON_RPC.ipynb">
      <img width="32px" src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Environment Setup

In [1]:
import os

# API_KEY = '' # @param {type:"string"}
# os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE" # Use Vertex AI API
# os.environ["GOOGLE_API_KEY"] = API_KEY

PROJECT_ID = "hupili-genai-bb"  # @param {type:"string"}
if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1" # @param {type:"string"}

STAGING_BUCKET = 'gs://agent_engine_deploy_staging' # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API

# [your-project-id]

In [2]:
import vertexai

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
!pip install google-adk==1.5.0

In [5]:
from google import adk

In [6]:
adk.__version__

'1.5.0'

In [7]:
%%file utils.py

import json
import time

def pprint_events(events):
    '''Pretty print of events generated by ADK runner'''
    start_time = time.time()
    for _, event in enumerate(events):
        is_final_response = event.is_final_response()
        function_calls = event.get_function_calls()
        function_responses = event.get_function_responses()
        agent_res = json.loads(event.content.model_dump_json(indent=2, exclude_none=True))

        if is_final_response:
            print('>>> inside final response...')
            final_response = event.content.parts[0].text
            end_time = time.time()
            elapsed_time_ms = round((end_time - start_time) * 1000, 3)
            print(f'Final Response ({elapsed_time_ms} ms):\n{final_response}')
            print("-----------------------------")
        elif function_calls:
            print('+++ inside the function call...')
            for function_call in function_calls:
                print(f"function, [args]:  {function_call.name}, {function_call.args}")
        elif function_responses:
            print('--- inside the function call response...')
            # TODO(Pili): copied from walkthrough codes. Find root cause of 'pending' not found.
            # if not event.actions.pending:
            if True:
                for function_response in function_responses:
                    details = function_response.response
                    recommended_list = list(details.values())
                    print(f"Function Name: {function_response.name}")
                    print(f"Function Results: {recommended_list}")
                    # result=json.dumps(recommended_list)
                    # print(f"Function Results {result}")
            else:
                print(agent_res)
    print(f"Total elapsed time: {elapsed_time_ms}")

import random
import asyncio

from google.adk.tools import google_search
from google.adk import Agent
from google.adk.agents import Agent, LlmAgent
from google.genai import types
from pydantic import BaseModel
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

async def caller_factory(root_agent, app_name='App12345', user_id='User12345', session_id=None):
  session_service = InMemorySessionService()
  if session_id is None:
    suffix = random.randint(100000, 999999)
    session_id = f'{app_name}-{user_id}-{suffix}'
  session = await session_service.create_session(
      app_name=app_name, user_id=user_id, session_id=session_id)
  runner = Runner(agent=root_agent, app_name=app_name, session_service=session_service)
  def _call(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=session.user_id, session_id=session.id, new_message=content)
    return events
  return _call

Overwriting utils.py


In [8]:
from utils import *

In [9]:
MODEL = "gemini-2.5-flash"
APP_NAME = "mcp_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "mcp_example_agent"

# Agent
simple_search_agent = Agent(
    model=MODEL,
    name=AGENT_NAME,
    description="Agent to answer questions using Google Search.",
    instruction="I can answer your questions by searching the internet. Just ask me anything!",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[google_search],
)

In [10]:
call_agent = await caller_factory(simple_search_agent)

In [11]:
events = call_agent('Hello')

In [12]:
pprint_events(events)

>>> inside final response...
Final Response (1297.741 ms):
Hello! How can I help you today?
-----------------------------
Total elapsed time: 1297.741


In [13]:
events = call_agent('What is the weather in Hong Kong now?')
pprint_events(events)

>>> inside final response...
Final Response (5450.438 ms):
The current weather in Hong Kong is light thunderstorms and rain with passing clouds. The temperature is around 86°F (30°C), but it feels like 96°F (36°C) due to high humidity, which is currently about 77%. The wind is blowing from the East at 9 mph. A Thunderstorm Warning has been issued and is effective until 1:00 PM.
-----------------------------
Total elapsed time: 5450.438


## Start with JSON RPC

The core if MCP and JSON-RPC are enclosed in the comments below.

In [14]:
%%file jsonrpc_server.py
import sys
import json

# MCP spec: A dummy initialize method
def initialize():
    capabilities = {}
    return {
        "capabilities": capabilities,
        "protocolVersion": "2025-06-18",
        "serverInfo": {
            "name": "A simple shim MCP server using stdio",
            "version": "1.2.3",
            "instructions": "The server provides information about a user"
        },
    }

# MCP spec: tools/list
def tools_list():
    """Returns a list of available tools."""
    # In a real MCP server, this would dynamically list registered tools.
    # For this simulation, we'll return a predefined list of the methods we handle.
    return {"tools": [
      {
        "name": "name",
        "title": "Get the name of the user",
        "description": "Get the name of the user",
        "inputSchema": {
          "type": "object",
          "properties": {
          },
          "required": []
        }
      }
    ]}


# This is our working function. Isolate it for unit test.
def name():
    """Returns a predefined name."""
    return "Pili"

# MCP spec: tools/call
def tools_call(func_name, arguments={}, **kwargs):
    # The structure is defined by MCP:
    # https://modelcontextprotocol.io/specification/2025-06-18/server/tools
    tool_call_response = {
        "content": None,
        "isError": False
    }
    if func_name == 'name':
        # Defined by MCP. A list of content parts.
        tool_call_response['content'] = [
            {
                "type": "text",
                "text": f'The user name is {name()}',
            }
        ]
    else:
        raise Exception(f"Tool '{name}' not found.")
    return tool_call_response

# JSON-RPC 2.0 spec
def handle_request(request):
    """Handles a single JSON-RPC request."""
    response = {"jsonrpc": "2.0"}
    request_id = request.get("id")
    response["id"] = request_id # Include the original ID in the response

    try:
        method = request.get("method")
        params = request.get("params", []) # Get parameters, default to empty list

        if method == "initialize":
            result = initialize()
            response["result"] = result
        elif method == "tools/list":
            result = tools_list()
            response["result"] = result
        elif method == "tools/call":
            result = tools_call(func_name=params.get('name', None), arguments=params.get('arguments', {}))
            response["result"] = result
        elif method == "name":
            # Assuming 'name' method doesn't require params for this simple example
            result = name()
            response["result"] = result
        else:
            # Method not found
            response["error"] = {
                "code": -32601,
                "message": f"Method not found: {method}"
            }

    except Exception as e:
        # Internal error during request handling
        response["error"] = {
            "code": -32603,
            "message": f"Internal error during request handling. Error: {e}. Method: {method}. Params: {params}",
            "data": str(e)
        }

    # Print the JSON response to standard output
    print(json.dumps(response))
    sys.stdout.flush() # Ensure the output is sent immediately

# Main loop to read from standard input line by line
if __name__ == "__main__":
    print("Simulated JSON-RPC server started. Waiting for requests on stdin...", file=sys.stderr)
    try:
        while True:
            line = sys.stdin.readline()
            if not line: # Empty line indicates EOF
                break
            try:
                request = json.loads(line)
                handle_request(request)
            except json.JSONDecodeError:
                # Handle invalid JSON on a line
                error_response = {
                    "jsonrpc": "2.0",
                    "id": None, # JSON-RPC spec suggests null for parse errors if ID is unknown
                    "error": {
                        "code": -32700,
                        "message": "Parse error: Invalid JSON was received on one line."
                    }
                }
                print(json.dumps(error_response))
                sys.stdout.flush()
            except Exception as e:
                 # Handle other errors during request processing for a line
                error_response = {
                    "jsonrpc": "2.0",
                    "id": None, # Or request.get("id") if available before the error
                    "error": {
                        "code": -32603,
                        "message": "Internal error during request processing for a line",
                        "data": str(e)
                    }
                }
                print(json.dumps(error_response))
                sys.stdout.flush()

    except Exception as e:
        raise
        # print(f"An unexpected error occurred in the main loop: {e}", file=sys.stderr)

    # print("Simulated JSON-RPC server shutting down.", file=sys.stderr)

Overwriting jsonrpc_server.py


### Example stdio requests of JSON-RPC

Check the request/response to understand the structure.

Note:

- request2.json is not a MCP request. However, it is a valid JSON-RPC request.

JSON-RPC response:

```
{"jsonrpc": "2.0", "id": 1, "result": |Any JSON value here|}
```

MCP further defines the `result` field:

```
{
    "content": [
      {
        "type": "text",
        "text": "Current weather in New York:\nTemperature: 72°F\nConditions: Partly cloudy"
      }
    ],
    "isError": false
}
```

In [15]:
%%file request0.json
{"jsonrpc": "2.0", "method": "initialize", "params": [], "id": 1}

Overwriting request0.json


In [16]:
!cat request0.json | python jsonrpc_server.py

Simulated JSON-RPC server started. Waiting for requests on stdin...
{"jsonrpc": "2.0", "id": 1, "result": {"capabilities": {}, "protocolVersion": "2025-06-18", "serverInfo": {"name": "A simple shim MCP server using stdio", "version": "1.2.3", "instructions": "The server provides information about a user"}}}


In [17]:
%%file request1.json
{"jsonrpc": "2.0", "method": "tools/list", "params": [], "id": 1}

Overwriting request1.json


In [18]:
!cat request1.json | python jsonrpc_server.py

Simulated JSON-RPC server started. Waiting for requests on stdin...
{"jsonrpc": "2.0", "id": 1, "result": {"tools": [{"name": "name", "title": "Get the name of the user", "description": "Get the name of the user", "inputSchema": {"type": "object", "properties": {}, "required": []}}]}}


In [19]:
%%file request2.json
{"jsonrpc": "2.0", "method": "name", "params": [], "id": 1}

Overwriting request2.json


In [20]:
!cat request2.json | python jsonrpc_server.py

Simulated JSON-RPC server started. Waiting for requests on stdin...
{"jsonrpc": "2.0", "id": 1, "result": "Pili"}


In [21]:
%%file request3.json
{"jsonrpc": "2.0", "method": "tools/call", "params": {"name": "name", "arguments": ""}, "id": 1}

Overwriting request3.json


In [22]:
!cat request3.json | python jsonrpc_server.py

Simulated JSON-RPC server started. Waiting for requests on stdin...
{"jsonrpc": "2.0", "id": 1, "result": {"content": [{"type": "text", "text": "The user name is Pili"}], "isError": false}}


In [23]:
import json
json.loads('''
{"jsonrpc": "2.0", "id": 1, "result": {"content": [{"type": "text", "text": "The user name is Pili"}], "isError": false}}
''')

{'jsonrpc': '2.0',
 'id': 1,
 'result': {'content': [{'type': 'text', 'text': 'The user name is Pili'}],
  'isError': False}}

### Connect into ADK

Note:

- MCPToolset tries to connect the stderr of our server script.
- However, CoLab environment mounts stderr to special handler.
- There is a conflict.

Towards this end, we save both agent.py and jsonrpc_server.py to file and run them through shell.

In [24]:
%%file agent.py

from google.adk.agents import Agent
from google.adk.tools.google_search_tool import GoogleSearchTool
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, StdioConnectionParams
from utils import *
import asyncio

name_tool = MCPToolset(
    connection_params=StdioConnectionParams(
        server_params=dict(
          command='python',
          args=[
              'jsonrpc_server.py',
          ],
        )
    )
)

root_agent = Agent(
    model='gemini-2.5-flash',
    name='root_agent',
    description='An agent that greet the user based on their name.',
    instruction='Greet the user based on their name',
    tools=[name_tool],
)

async def main():
  call_agent = await caller_factory(root_agent)
  events = call_agent('Hello')
  pprint_events(list(events))

if __name__ == '__main__':
  asyncio.run(main())

Overwriting agent.py


In [25]:
!python agent.py

Simulated JSON-RPC server started. Waiting for requests on stdin...
/usr/local/lib/python3.11/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:85: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
an error occurred during closing of asynchronous generator <async_generator object stdio_client at 0x7a4e24b78680>
asyncgen: <async_generator object stdio_client at 0x7a4e24b78680>
  + Exception Group Traceback (most recent call last):
  |   File "/usr/local/lib/python3.11/dist-packages/anyio/_backends/_asyncio.py", line 772, in __aexit__
  |     raise BaseExce